In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import requests
from time import strftime
import datetime
import time
from pandas import ExcelWriter
import re
import pyodbc
import asyncio
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pdb
#pdb.set_trace()
from ipywidgets import IntProgress
from IPython.display import display
import os
from selenium.webdriver.support import expected_conditions
from IPython.display import clear_output

In [2]:
TIEMPO_DE_ESPERA = 5.5

start_ = datetime.datetime.now()
fecha = datetime.date.today()
hoy = fecha.strftime('%Y/%m/%d')

user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--window-size=1825x1844")
# options.headless = True
browser = webdriver.Chrome( options = chrome_options)


In [3]:
URLS = [['Mujer','https://www2.hm.com/content/hmonline/en_us/women/products/view-all.html/'],
       ['Hombre','https://www2.hm.com/en_us/men/products/view-all.html'],
       ['Kids','https://www2.hm.com/en_us/kids/products/view-all.html']]

In [4]:
#
paginacion_total = []
for URL in URLS:
    browser.get(URL[1])
    
    if URL[0] =='Kids':
        paginacion_total += browser.execute_script('''

            var cantidad = document.querySelector("#menu-links > li:nth-child(3)").getElementsByClassName('item').length;

            var arn =[]
            var arl =[]

            for (i=0;i<cantidad;i++){


                arn.push(document.querySelector("#menu-links > li:nth-child(3)").getElementsByClassName('item')[i].textContent.trim());

                arl.push('https://www2.hm.com'.concat(document.querySelector("#menu-links > li:nth-child(3)").getElementsByClassName('item')[i].getElementsByTagName('a')[0].getAttribute('href')));
            }
             function zip(a, b) {
                              var arr = [];
                              for (var key in a) arr.push([a[key], b[key]]);
                              return arr;
                            }
            return zip(arl,arn);

            ''')
    else:
        
        posicion = 4    
        paginacion_total += browser.execute_script('''

        var cantidad = document.querySelector("#menu-links > li:nth-child('''+str(posicion)+''')").getElementsByClassName('item').length;

        var arn =[]
        var arl =[]

        for (i=0;i<cantidad;i++){


            arn.push(document.querySelector("#menu-links > li:nth-child('''+str(posicion)+''')").getElementsByClassName('item')[i].textContent.trim());

            arl.push('https://www2.hm.com'.concat(document.querySelector("#menu-links > li:nth-child('''+str(posicion)+''')").getElementsByClassName('item')[i].getElementsByTagName('a')[0].getAttribute('href')));
        }
         function zip(a, b) {
                          var arr = [];
                          for (var key in a) arr.push([a[key], b[key]]);
                          return arr;
                        }
        return zip(arl,arn);

        ''')

for paginacion in paginacion_total:
    
    if paginacion[0].find('/men') != -1 : paginacion.append('Men')
    if paginacion[0].find('/kids') != -1 : paginacion.append('Kids')
    if paginacion[0].find('/women') != -1 : paginacion.append('Women')
        

In [8]:
dfLinks = pd.DataFrame(data=paginacion_total,columns=['url',
                                                      'tipo',
                                                      'sexo'])

In [9]:
dfLinks.to_excel(f'dfLinks_{str(fecha)}.xlsx')

In [10]:
browser.quit()